<a href="https://colab.research.google.com/github/cesarofuchi/python_projects/blob/main/TabelaSanepar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Relatório com previsão de rodizio de água para sua região em Curitiba</h1>

A seca pelo qual Curitiba e região metropolitana esta passando pode estar longe de acabar. A Sanepar tem feito um esforço para divulgar o META 20 para economizarmos 20% de água e assim diminuirmos o tempo de rodízio. Os rodízios de água mudam de acordo com os níveis de água dos reservatórios. No meu condomínio o rodízio é avisado no dia por whatsapp, e se você for como eu, que não se informa como deveria, tem que ficar revisando as mensagens para ter certeza de quando começa e quando termina. 

Pelo site da [Sanepar](http://site.sanepar.com.br/mapa-rodizio-abastecimento-curitiba-rmc) é possível verificar pelo endereço como está a previsão do abastecimento em sua região. Porém ele só informa quando será o próximo rodízio.
 
![picture](https://github.com/cesarofuchi/python_projects/blob/main/sanepar/img/banner_20porcento.jpg?raw=true)

## Obtendo os dados
Acontece que na [agência estadual de notícias](http://www.aen.pr.gov.br/)  existe uma [tabela](http://www.aen.pr.gov.br/arquivos/0910rodizioatualizada.pdf) mais detalhada, com uma previsão maior das datas de rodízio. 

![picture](https://github.com/cesarofuchi/python_projects/blob/main/sanepar/img/Tabela.JPG?raw=true)

A tabela possui as datas de início e término do rodízio, quais os bairros afetados e o Grupo do rodízio. Em Curitiba, atualmente temos 3 grupos, mas isso já mudou antes também. 

Um mesmo bairro pode ter dias diferentes de rodízio, então é preciso se informar qual o seu reservatório de abastecimento, fazendo uma procura pelo bairro e pelas datas de início e término do rodízio (pode conferir pelo site da sanepar isso). 

Feito isso podemos usar o identificador da primeira coluna ou o nome da sua área (na penúltima coluna) para saber qual a sua micro-região. 

Exemplo: RISO - (Área do Recalque do Reservetório Iguaçu)

Munidos dessa informação vamos extrair as datas da sua região e gerar um relatório em PDF para você imprimir, deixar no elevador, na geladeira, ou simplemente deixar no celular para avaliar as datas em que faltará água na sua casa.

Primeiramente devemos fazer o download da tabela para deixar aqui na maquina virtual. Esse link irá mudar de tempos em tempo, então procure a mais atualizada.

Vou usar a biblioteca requests para salvar o arquivo na maquina local.

In [57]:
import requests
#import textract
response = requests.get("http://www.aen.pr.gov.br/arquivos/0910rodizioatualizada.pdf")
#myfile = open("out.pdf", "w")

with open('document.pdf', 'wb') as fw:
    fw.write(response.content)


A tabela está salva como "document.pdf"

## Extração dos dados da tabela em PDF


Para extrar e manipular os dados da tabela será utilizada a biblioteca de Python [PDF Plumber](https://github.com/jsvine/pdfplumber). Ela possui uma grande gama de funções e argumentos, e aqui nós vamos começar a nos aventurar com essas funções.

Lembrando: Nativamente, o Google Colab não possui essa biblioteca instalada, então nós precisamos instalar a mesma no nosso ambiente.

Para isso, vamos utilizar o gerenciador de pacotes pip, e com o comando install vamos instalar a biblioteca.

In [5]:
!pip install pdfplumber

     |████████████████████████████████| 5.6MB 6.2MB/s 
     |████████████████████████████████| 143kB 38.4MB/s 
     |████████████████████████████████| 13.7MB 262kB/s 
  Created wheel for pdfplumber: filename=pdfplumber-0.5.23-cp36-none-any.whl size=29176 sha256=a6e4d55a00babdefbdc328c0a2b0ef653512f7ff98fd15cb06d0dd8a2d24fb41
  Stored in directory: /root/.cache/pip/wheels/f6/4d/ae/abb7b92fbed5d09a835c259f26d2a3223ea929a0ad6d322097
Successfully built pdfplumber


### pdfplumber.open()

Para abrir um arquivo, vamos associá-lo a uma variável, para que ele possa ser facilmente acessado depois.

Utilizamos o comando `pdfplumber.open('caminho/do/meu/arquivo')` com o caminho do arquivo dentro dos parênteses, dentro de aspas, simples ou duplas.

In [58]:
#biblioteca para manipular arquivos
import pdfplumber

In [64]:
#carregar a tabela em pdf
pdf = pdfplumber.open("document.pdf")

### pages
Para acessar as páginas existentes no arquivo, utilizamos a função `pages`.

Ela retorna uma lista com as páginas do arquivo.

Perceba que a tabela de rodízio possui muitas páginas

In [65]:
pdf.pages

[<Page:1>,
 <Page:2>,
 <Page:3>,
 <Page:4>,
 <Page:5>,
 <Page:6>,
 <Page:7>,
 <Page:8>,
 <Page:9>,
 <Page:10>,
 <Page:11>,
 <Page:12>,
 <Page:13>,
 <Page:14>,
 <Page:15>]

In [66]:
pdf.metadata

{'Author': 'Edymilson Luiz Dos Santos',
 'CreationDate': "D:20201009075221-03'00'",
 'Creator': 'Microsoft® Excel® 2013',
 'ModDate': "D:20201009075221-03'00'",
 'Producer': 'Microsoft® Excel® 2013'}

### extract_text()
Para extrair um texto de um pdf utilizamos a função `extract_text`

Ela retorna o texto em formato string para buscar a informação. Em geral esse processa implica em separar as linhas, procurar por expressões regulares 'regex', etc.

In [61]:
# abrir a primeira pagina do documento
pagina = pdf.pages[0]
texto = pagina.extract_text()
texto

'09 de Outubro de 2020 - Sexta-feira NOVO GRUPO 3\nCODOPE INICIADO EM Das Término Normalização Bairros afetados Grupo\nCuritiba (área da Gravidade do Reservatório Corte Branco): Guabirotuba, Uberaba, Alto \nGCBR 00030... 09/10/2020 16:00 10/10/2020 16:00 11/10/2020 04:00 Boqueirão, Boqueirão, Hauer. 3\nCuritiba (Área do Recalque do Reservatório Corte Branco): Cajuru, Guabirotuba, Jardim das \nRCBR 00031 09/10/2020 16:00 10/10/2020 16:00 11/10/2020 04:00 Américas, Uberaba. 3\nCuritiba (Área da Gravidade do Reservatório Cajuru): Jardim Botânico, Rebouças, Centro, \nCristo Rei, Hugo Lange, Prado Velho,  Juvevê, Alto da XV.\nGCAJ 00040 09/10/2020 16:00 10/10/2020 16:00 11/10/2020 04:00 3\nPiraquara (área da Gravidade do Reservatório Guarituba Redondo): Planta Pontoni, Planta \nRicardo Vagner, Jardim Alterosa, Jardim Itiberê, Jardim Dos Eucaliptos, Bosque Centenário, \nPlanta Guarituba, Bosque Tarumã, Vila Izabel, Vila Palmas, Vila Dirce, Vila Pedro Alcântara, \nGGRE 00560... 09/10/2020 16:

###extract_table()
Para o caso de planilhas em pdf (como é o caso da [tabela](http://www.aen.pr.gov.br/arquivos/0910rodizioatualizada.pdf) de rodízio) existe uma função mais apropriada chamada `extract_table`, que retorna os dados em formato de lista

In [62]:
tabela=pdf.pages[0].extract_table()
tabela

[['09 de Outubro de 2020 - Sexta-feira NOVO GRUPO 3',
  None,
  None,
  None,
  None,
  None,
  ''],
 ['',
  'CODOPE INICIADO EM',
  'Das',
  'Término',
  'Normalização',
  'Bairros afetados',
  'Grupo'],
 ['GCBR',
  '00030...',
  '09/10/2020 16:00',
  '10/10/2020 16:00',
  '11/10/2020 04:00',
  'Curitiba (área da Gravidade do Reservatório Corte Branco): Guabirotuba, Uberaba, Alto \nBoqueirão, Boqueirão, Hauer.',
  '3'],
 ['RCBR',
  '00031',
  '09/10/2020 16:00',
  '10/10/2020 16:00',
  '11/10/2020 04:00',
  'Curitiba (Área do Recalque do Reservatório Corte Branco): Cajuru, Guabirotuba, Jardim das \nAméricas, Uberaba.',
  '3'],
 ['GCAJ',
  '00040',
  '09/10/2020 16:00',
  '10/10/2020 16:00',
  '11/10/2020 04:00',
  'Curitiba (Área da Gravidade do Reservatório Cajuru): Jardim Botânico, Rebouças, Centro, \nCristo Rei, Hugo Lange, Prado Velho,  Juvevê, Alto da XV.',
  '3'],
 ['GGRE',
  '00560...',
  '09/10/2020 16:00',
  '10/10/2020 16:00',
  '11/10/2020 04:00',
  'Piraquara (área da Grav

A próxima etapa será juntar todas as páginas para depois extrairmos todas as datas de rodízio para a nossa região. 
O código a seguir concatena essas listas na variável table

In [24]:
#iterar por todas as paginas e concatenar as tabelas
table=[]
for i in range(len(relatorio.pages)):  
  pagina=relatorio.pages[i]
  table=table+pagina.extract_table() 
  

Para extrair somente as datas da região escolhida, podemos usar a primeira coluna que é o código `coluna=0` ou a sexta coluna `coluna=5` se usarmos toda a string da área/reservatório e comparar com a string da nossa região `str_procura`. É preciso descomentar qual o conjunto escolhido.

O código separa as datas de início e fim do rodizio da região na variável `resultado`

In [160]:
# escolher método de procura pela sua região

#procura por codigo
#str_procura='RBBC'
#coluna=0

#procura por área / reservatório
str_procura="Área da Gravidade do Reservatório Campo Comprido"
coluna=5

resultado=[]

for i in range(len(table)):
  if table[i][coluna] is not None:
    if table[i][coluna].find(str_procura)>=0:
      # resultado encontrado, separar os dados de inicio e fim do rodizio
      resultado.append([table[i][2],table[i][4]])
      # salvando o local para o relatório
      textoLocal=table[i][5]
      print('inicio=',table[i][2],' fim=', table[i][4], textoLocal)
      
#adicionando nomes para as colunas
resultado.insert(0,['Início','Fim'])


inicio= 11/10/2020 16:00  fim= 13/10/2020 04:00 Curitiba (Área da Gravidade do Reservatório Campo Comprido): Campo Comprido, Cidade 
Industrial, Mossunguê.
inicio= 14/10/2020 16:00  fim= 16/10/2020 04:00 Curitiba (Área da Gravidade do Reservatório Campo Comprido): Campo Comprido, Cidade 
Industrial, Mossunguê.
inicio= 17/10/2020 16:00  fim= 19/10/2020 04:00 Curitiba (Área da Gravidade do Reservatório Campo Comprido): Campo Comprido, Cidade 
Industrial, Mossunguê.


## Gerando um relatório em PDF
Com os resultados em mãos, vamos gerar um relatório em pdf para divulgar o resultado.

Para gerar um relatório em pdf será usada a biblioteca [reportlab](https://www.reportlab.com/). Vamos usar o mínimo de elementos nesse código, um texto, uma tabela e uma imagem. Para mais detalhes da biblioteca vejam o [manual de uso](https://www.reportlab.com/docs/reportlab-userguide.pdf). 

A biblioteca também não é padrão no Colab, então precisamos adiciona-la ao nosso projeto com `!pip install reportlab`






In [17]:
!pip install reportlab

     |████████████████████████████████| 2.6MB 2.7MB/s 


Adicionando elementos que vamos utilizar da biblioteca:

In [105]:
from reportlab.pdfgen import canvas
from reportlab.platypus import Frame, Image
from reportlab.lib.units import cm
from reportlab.platypus import Paragraph, Frame, Table, Spacer, TableStyle
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.styles import ParagraphStyle

### Entendendo a estrutura do reportlab utilizada
Para melhor entender a estrutura de código para gerar o relatório veja a figura a seguir.

O `Canvas` é uma tela A4 (com uma margem interna)

Vamos adicionar um `Frame` que é um container de elementos nesse `Canvas` a 3 cm de distância da margem esquerda e a 12 cm de distância da margem inferior. As distãncias são relativas a posição (0,0) no canto inferior esquedo.


Dentro do `Frame` montamos uma `Story`, que é uma sequência de elementos: 

- Parágrafo 1: texto padrão
- Parágrafo 2: criada a partir do texto da região procurada
- Tabela: criada a partir da tabela de resultados 
- Imagem: criada a partir de um link

Foram adicionados espaçamentos (`Spacer`) entre os elementos

![picture](https://github.com/cesarofuchi/python_projects/blob/master/sanepar/img/reportlab.JPG?raw=true)







### Código
Deixei o código inteiro para que seja mais fácil editar as posições e estilos dos elementos, e gerar um pdf novo. Assim pode-se aprender na prática o que cada mudança nos elementos provoca.

Ao final do processo é gerado um pdf com informações sobre a sua região e o horário de início e fim do rodízio.

In [163]:
#criação do Canvas
nome_pdf = "rodizio_sanepar.pdf" 
c = canvas.Canvas(nome_pdf)
c.setTitle(nome_pdf)

#Criação da tabela a partir do resultado
rtable=Table(resultado)
 # propriedades da tabela
t_fontsize=16
t_leading=table_size+2

rtable.setStyle(TableStyle([
  ('LEADING', (0, 0), (-1, -1), t_leading),
  ('SIZE', (0, 0), (-1, -1), t_fontsize),
  ('INNERGRID', (0, 0), (-1, -1), 0.25, colors.black),
  ('BOX', (0, 0), (-1, -1), 0.25, colors.black)
]))
###############################################################
# padrão é formato A4 (21 cm)
# posição em CM, posição (0,0) canto inferior esquerdo

# estilo do paragrafo 1        
titleFormat = '<font size="18" name="Helvetica" color="rgb(0, 89, 164)"><b>%s</b></font>'
style = getSampleStyleSheet()['Normal']
style.leading = 24

# paragrafo 1
p1 = Paragraph(titleFormat % 'RODÍZIO DE ABASTECIMENTO', style)

# estilo do paragrafo 2
titleFormat = '<font size="16" name="Helvetica" color=black><i>%s</i></font>'
# paragrafo 2 
p2 = Paragraph(titleFormat % textoLocal, style)

# imagem 
im=Image("https://github.com/cesarofuchi/python_projects/blob/main/sanepar/img/banner_20porcento.jpg?raw=true")
# restrição de tamanho para reduzir a imagem de forma proporcional
im._restrictSize(10 * cm, 12 * cm)
im.hAlign = 'RIGHT'

# story com os elementos espaçados        
story = [p1,
         Spacer(1, 10),
         p2,
         Spacer(1, 10),
         rtable,
         Spacer(10, 30),
         im]

# criação do Frame
f = Frame(3*cm, 12*cm, 16 * cm, 16 * cm)
# adicionando o Frame (com Story) ao Canvas
f.addFromList(story,c)

# salvando no diretório local
c.save()
print('{}.pdf criado com sucesso!'.format(nome_pdf))

rodizio_sanepar.pdf.pdf criado com sucesso!
